In [ ]:
# !pip install langchain
# !pip install pypdf2
# !pip install langchain-community 
# !pip install unstructured
# !pip install "unstructured[pdf]"
# !apt-get install poppler-utils
# !apt-get install tesseract-ocr
# !apt-get install libtesseract-dev

# !pip install faiss-cpu
# !pip install sentence-transformers

# !pip install langchain-huggingface

# !pip install huggingface_hub

# !pip install transformers

# !pip install accelerate

# !pip install bitsandbytes

# !pip install python-dotenv




In [ ]:
from langchain_community.document_loaders import DirectoryLoader
Data_path = "C:/Users/srajp/OneDrive/Desktop/GenAI/ChatBot/Content/"
def load_documents(Data_path):
  loader = DirectoryLoader(Data_path,glob="**/*.pdf")
  documents = loader.load()
  return documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pickle
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,length_function=len,add_start_index=True)
documents = load_documents(Data_path)
print(len(documents))
texts = text_splitter.split_documents(documents)
with open('texts.pkl', 'wb') as f:
    pickle.dump(texts, f)
print(len(texts))

In [1]:
##import locally stored split data
import pickle
text_Path = "split_data/texts.pkl"
with open(text_Path, 'rb') as f:
    texts1 = pickle.load(f)
print(len(texts1))


723


In [2]:
print(texts1[1].page_content)
print("Metadata -",texts1[0].metadata)

FOUNDATIONS OF SOFTWARE TESTING

ISTQB CERTIFICATION

Dorothy Graham

Erik van Veenendaal

Isabel Evans

Rex Black

CONTENTS

Acknowledgements viii Preface ix

1 Fundamentals of testing 1

1.1 Why is testing necessary? 1 1.2 What is testing? 11 1.3 Testing principles 18 1.4 Fundamental test process 20 1.5 The psychology of testing 26 Chapter review 31 Sample exam questions 32 Exercise: Test psychology 33 Exercise solution 34

2 Testing throughout the software life cycle 35

2.1 Software development models 35 2.2 Test levels 41 2.3 Test types: the targets of testing 46 2.4 Maintenance testing 50 Chapter review 54 Sample exam questions 55

3 Static techniques 57 3.1 Reviews and the test process 57

3.2 Review process 59 3.3 Static analysis by tools 69 Chapter review 74 Sample exam questions 75

4 Test design techniques 77
Metadata - {'source': '/content/sample_data/Foundations of software testing.pdf', 'start_index': 0}


In [ ]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# Replace with the desired model name
model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Create FAISS vectorstore
db = FAISS.from_documents(texts, embeddings)
db.save_local("Vector_db/faiss_index")

In [ ]:
# To load locally store vector_db back
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# Replace with the desired model name
model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
db1 = FAISS.load_local("Vector_db/faiss_index", embeddings, allow_dangerous_deserialization=True) # Set allow_dangerous_deserialization to True

In [ ]:
question = "Importance ISTQB Certificate in IT?"
results = db1.similarity_search_with_relevance_scores(question,k=3)
print(results)

In [ ]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=question)
print(prompt)

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
secKey = os.environ.get("HUGGINGFACEHUB_API_TOKEN")
# print(secKey)

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the API token
api_token = os.environ.get("HUGGINGFACEHUB_API_TOKEN")

if api_token is None:
    print("Error: HUGGINGFACEHUB_API_TOKEN not found in .env file.")
else:
    # Use the API token here
    print(f"API token retrieved: {api_token[:5]}... (truncated for security)")


In [ ]:
from os import environ
environ['HUGGINGFACEHUB_API_TOKEN'] = secKey

In [ ]:
# !pip install langchain_huggingface
from langchain_huggingface import HuggingFaceEndpoint

repo_id ="mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceEndpoint(repo_id=repo_id, temperature=0.1, max_length=512, token=secKey) # Pass temperature and max_length as separate arguments

In [ ]:
from pprint import pprint
output = llm.invoke(prompt)
# print(output)
cleaned_output = output.replace("'", "").replace("\n", " ")
print(cleaned_output)